In [0]:
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import glob

# Make sure you have access to the images' folders
drive.mount('/content/drive/')

Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
cd drive/Team Drives

/content/drive/Team Drives


In [0]:
cd ECENproject2

[Errno 2] No such file or directory: 'ECENproject2'
/content/drive/Team Drives/ECENproject2


In [0]:
####The Convolutional Neural Network for stitched images####
#----------------------------------------------------------#

# Still requires tweaking. Also keep it simple for faster forward\backward propagations
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape= (320, 180, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [0]:
####       Running the CNN          ####
#--------------------------------------#

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

#history = model.fit(X_train, y_train, epochs = 30, batch_size = 200, verbose = 2)

#loss  = model.evaluate(X_test, y_test)
#predictions = model.predict(X_test)

#model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 318, 178, 16)      160       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 159, 89, 16)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 157, 87, 16)       2320      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 78, 43, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 76, 41, 16)        2320      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 38, 20, 16)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 38, 20, 16)        0         
__________

In [0]:
nb_epochs = 50
batch_size = 15
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    validation_split=.05)

train_generator = train_datagen.flow_from_directory(
    'monoscopic',
    target_size=(320, 180),#width , height 
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    'monoscopic',
    target_size=(320, 180),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    subset='validation') # set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

model.save("MB_monoscopic_unseen_stitched_result.h5")

Found 2640 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Epoch 1/50
176/176 [==============================] - 33s 190ms/step - loss: 0.4115 - acc: 0.7902 - val_loss: 0.3464 - val_acc: 0.8296
Epoch 2/50
176/176 [==============================] - 31s 177ms/step - loss: 0.1137 - acc: 0.9602 - val_loss: 0.5918 - val_acc: 0.6748
Epoch 3/50
176/176 [==============================] - 32s 184ms/step - loss: 0.0585 - acc: 0.9780 - val_loss: 0.6596 - val_acc: 0.7398
Epoch 4/50
176/176 [==============================] - 31s 178ms/step - loss: 0.0440 - acc: 0.9852 - val_loss: 0.3973 - val_acc: 0.8049
Epoch 5/50
176/176 [==============================] - 32s 182ms/step - loss: 0.0284 - acc: 0.9905 - val_loss: 0.7461 - val_acc: 0.7398
Epoch 6/50
176/176 [==============================] - 32s 180ms/step - loss: 0.0253 - acc: 0.9924 - val_loss: 0.9504 - val_acc: 0.7073
Epoch 7/50
176/176 [==============================] - 32s 182ms/step - loss: 0.0286 - acc: 0.9917 - val_los

# GOOD MODELS